In [1]:
import pandas as pd
import json
import ast

In [2]:
#Cargamos los datos.
data = []
with open(r'Dataset\australian_users_items.json', 'r', encoding='utf-8') as file:
    for line in file:
        data.append(eval(line))

In [3]:
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [4]:
df.shape


(88310, 5)

In [5]:
#Elimino los valores nulos
df = df.dropna()
df.shape

(88310, 5)

In [6]:
df.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [7]:
f_row = df.iloc[0]['items']
print(f_row)

[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever': 1853, 'playtime_2weeks': 0}, {'item_i

In [8]:
#Creo un nuevo df con las columnas que nesecito.
df1 = df[['user_id','items']].copy()
df1.head(3)

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [9]:
# tengo que crear una dataset donde extraiga del diccionari de la columna items 'item_id' y 'playtime_forever'

dfe = df1.explode('items')
dfe = dfe.reset_index(drop=True)
df_items = pd.json_normalize(dfe['items'])


In [10]:
df_items.head(5)       

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0


In [11]:
# Concateno los dataframe . En esta parte identificamos los items con los items de usuarios_id y 
#creo un nuevo dataframe ya con la columna user_id incluida.
df_result = pd.concat([dfe[['user_id']], df_items], axis=1)
df_result

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...
5170010,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,519140,Minds Eyes,3.0,3.0


In [12]:
df_i = df_result.drop('playtime_2weeks', axis=1).copy()
df_i

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
5170010,76561198329548331,373330,All Is Dust,0.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0
5170013,76561198329548331,519140,Minds Eyes,3.0


### Cargo el archivo 'australian_user_reviews.json' para crear un df con las columnas 'user_id' y 'review_item_id'


In [13]:
data = []
with open(r'Dataset\australian_user_reviews.json', 'r', encoding='utf-8') as file:
    for line in file:
        data.append(eval(line))

In [14]:
df_u = pd.DataFrame(data)
df_u.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [15]:
df_u.head(3)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


In [16]:
# Saco los valores anidados en la columna review
df_review_ex = df_u.explode('reviews')
df_review_ex = df_review_ex.reset_index(drop=True)
df_reviews = pd.json_normalize(df_review_ex['reviews'])

df_re_2 = pd.concat([df_review_ex[['user_id']], df_reviews], axis=1)

df_re_2

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59328,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [17]:
df_re_2 = df_re_2.drop(['funny', 'posted', 'last_edited', 'helpful', 'recommend', 'review'], axis=1)

In [18]:
df_re_2.head(3)

,user_id,item_id
0,76561197970982479,1250
1,76561197970982479,22200
2,76561197970982479,43110


In [19]:
df_re_2 = df_re_2.rename(columns={'item_id': 'reviewed_item_id'})
df_re_2.head(3)

,user_id,reviewed_item_id
0,76561197970982479,1250
1,76561197970982479,22200
2,76561197970982479,43110


In [20]:
review_sum_df = df_re_2.groupby('user_id')['reviewed_item_id'].nunique().reset_index(name='review_sum')

In [21]:
review_sum_df

,user_id,review_sum
0,--000--,1
1,--ace--,2
2,--ionex--,2
3,-2SV-vuLB-Kg,5
4,-Azsael-,1
...,...,...
25480,zwanzigdrei,1
25481,zy0705,1
25482,zynxgameth,1
25483,zyr0n1c,9


In [22]:
final_df = pd.merge(review_sum_df, df_re_2, on='user_id', how='left')
final_df.head(5)

,user_id,review_sum,reviewed_item_id
0,--000--,1,1250
1,--ace--,2,440
2,--ace--,2,113200
3,--ionex--,2,730
4,--ionex--,2,105600


In [23]:
final_df = final_df.drop(columns='reviewed_item_id', axis=1)
final_df.head(3)

,user_id,review_sum
0,--000--,1
1,--ace--,2
2,--ace--,2


In [24]:
df_m1 = pd.merge(final_df, df_i, on='user_id', how='left')

In [25]:
df_m1.shape

(7817976, 5)

In [26]:
df_m1.head(20)

,user_id,review_sum,item_id,item_name,playtime_forever
0,--000--,1,4000,Garry's Mod,925.0
1,--000--,1,1250,Killing Floor,2949.0
2,--000--,1,35420,Killing Floor Mod: Defence Alliance 2,0.0
3,--000--,1,550,Left 4 Dead 2,2237.0
4,--000--,1,223530,Left 4 Dead 2 Beta,0.0
5,--000--,1,43110,Metro 2033,0.0
6,--000--,1,105600,Terraria,1172.0
7,--000--,1,205790,Dota 2 Test,0.0
8,--000--,1,209270,Hero Academy,38.0
9,--000--,1,104900,ORION: Prelude,63.0


### Cargo el archivo 'output_steam_games' para hacer un df con las columnas 'price', 'genres' y hacer un merge con la columna item_id.

In [27]:
data = []
with open(r'Dataset\\output_steam_games.json', 'r', encoding='utf-8') as file:
    for line in file:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError:
            continue  # Skip the line if it can't be parsed

In [28]:
dff = pd.DataFrame(data)
dff

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [29]:
dff1 = dff[['genres','price','id']].copy()
dff1.head(4)

,genres,price,id
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN


In [30]:
dff1.describe()

,genres,price,id
count,28852,30758.00,32133
unique,883,162.00,32132
top,[Action],4.99,612880
freq,1880,4278.00,2


In [31]:
dff1 =dff1.dropna()

In [32]:
de = dff1.explode('genres')
de_dum = pd.get_dummies(de['genres'], dtype='int')

In [33]:
dee = pd.concat([dff1,de_dum.groupby(level=0).sum()], axis=1)

In [34]:
dee = dee.drop('genres', axis=1)
dee.head(3)

,price,id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
88310,4.99,761140,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
88311,Free To Play,643980,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
88312,Free to Play,670290,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0


In [35]:
dee = dee.rename(columns={'id':'item_id'})

In [36]:
dee1 = dee.pop('item_id')
dee2 = pd.concat([dee, dee1], axis=1)
dee2.head(5) 

,price,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,item_id
88310,4.99,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,761140
88311,Free To Play,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,643980
88312,Free to Play,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,670290
88313,0.99,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,767400
88315,3.99,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,772540


In [37]:
#hago el mismo porcedimiento pero para pasar item_id al extremo izquierdo del df
df_m2 = df_m1.pop('item_id')

df_m1.insert(0,'item_id',df_m2)

In [38]:
df_m1.insert(0,'item_id',df_m2)

ValueError: cannot insert item_id, already exists

In [39]:
df_m1.head(3)

,item_id,user_id,review_sum,item_name,playtime_forever
0,4000,--000--,1,Garry's Mod,925.0
1,1250,--000--,1,Killing Floor,2949.0
2,35420,--000--,1,Killing Floor Mod: Defence Alliance 2,0.0


In [40]:
df_ready = pd.merge(dee2, df_m1, on='item_id', how='outer')

In [41]:
df_ready.shape

(7837214, 28)

In [42]:
#Vuelvo todos los valores numericos 
df_ready['price'] = pd.to_numeric(df_ready['price'], errors='coerce')

In [43]:
#Relleno los valos na creados con el valor de 0
df_ready['price'] = df_ready['price'].fillna(0)

In [44]:
df_ready.to_parquet(r'Dataset\Procesado\user_db.parquet', index=False)